<a href="https://colab.research.google.com/github/sunyux/Color-Transfer-between-Images/blob/main/CV_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import sys
import matplotlib.pyplot as plt

In [ ]:
def helper3by3(matrix, img):

    # apply 3by3 matrix to img vector 
    newimg = np.zeros_like(img,dtype=np.float32)
    newimg[...,0] = img[...,0]* matrix[0][0] +  img[...,1]* matrix[0][1] + img[...,2]* matrix[0][2]
    newimg[...,1] = img[...,0]* matrix[1][0] +  img[...,1]* matrix[1][1] + img[...,2]* matrix[1][2]
    newimg[...,2] = img[...,0]* matrix[2][0] +  img[...,1]* matrix[2][1] + img[...,2]* matrix[2][2]

    return newimg

In [ ]:

def convert_color_space_BGR_to_RGB(img_BGR):
    img_RGB = np.zeros_like(img_BGR,dtype=np.float32)
    img_RGB=img_BGR[...,::-1]
    return img_RGB

In [ ]:
def convert_color_space_RGB_to_BGR(img_RGB):
    img_BGR = np.zeros_like(img_RGB,dtype=np.float32)
    img_BGR=img_RGB[...,::-1]
    return img_BGR

In [ ]:
def convert_color_space_RGB_to_Lab(img_RGB):
    '''
    convert image color space RGB to Lab
    '''
    img_LMS = np.zeros_like(img_RGB,dtype=np.float32)
    LMS_RGB=np.array([[0.3811,0.5783,0.0402],[0.1967,0.7244,0.0782],[0.0241,0.1288,0.8444]])
    img_LMS[...,0] = np.log10(0.3811*img_RGB[:, :, 0] + 0.5783* img_RGB[:, :, 1]+ 0.0402*img_RGB[:, :, 2]) 
    img_LMS[...,1] =  np.log10(0.1967*img_RGB[:, :, 0] + 0.7244*img_RGB[:, :, 1] + 0.0782*img_RGB[:, :, 2]) 
    img_LMS[...,2] = np.log10(0.0241*img_RGB[:, :, 0] + 0.1288*img_RGB[:, :, 1] + 0.8444*img_RGB[:, :, 2])

    img_Lab = np.zeros_like(img_RGB,dtype=np.float32)
    Lab_LMS1=np.array([[1/np.sqrt(3),0,0],[0,1/np.sqrt(6),0],[0,0,1/np.sqrt(2)]])
    Lab_LMS2=np.array([[1,1,1],[1,1,-2],[1,-1,0]])
    Lab_LMS=np.matmul(Lab_LMS1,Lab_LMS2)
    img_Lab=helper3by3(Lab_LMS,img_LMS)
    return img_Lab



In [ ]:
def convert_color_space_Lab_to_RGB(img_Lab):
    '''
    convert image color space Lab to RGB
    '''
    img_LMS = np.zeros_like(img_Lab,dtype=np.float32)
    LMS_Lab1=np.array([[1,1,1],[1,1,-1],[1,-2,0]])
    LMS_Lab2=np.array([[1/np.sqrt(3),0,0],[0,1/np.sqrt(6),0],[0,0,1/np.sqrt(2)]])
    LMS_Lab=np.matmul(LMS_Lab1,LMS_Lab2)
    img_LMS=helper3by3(LMS_Lab, img_Lab)

    img_RGB = np.zeros_like(img_Lab,dtype=np.float32)
    RGB_LMS=np.array([[4.4679,-3.5873,0.1193],[-1.2186,2.3809,-0.1624],[0.0497,-0.2439,1.2045]])
    img_LMS = np.power(10, img_LMS)
    img_RGB = helper3by3(RGB_LMS, img_LMS)
    return img_RGB

In [ ]:
def convert_color_space_RGB_to_CIECAM97s(img_RGB):
    '''
    convert image color space RGB to CIECAM97s
    '''
    img_CIECAM97s = np.zeros_like(img_RGB,dtype=np.float32)
    img_LMS = np.zeros_like(img_RGB,dtype=np.float32)
    img_LMS[...,0] = np.log10(0.3811*img_RGB[:, :, 0] + 0.5783* img_RGB[:, :, 1]+ 0.0402*img_RGB[:, :, 2]) 
    img_LMS[...,1] =  np.log10(0.1967*img_RGB[:, :, 0] + 0.7244*img_RGB[:, :, 1] + 0.0782*img_RGB[:, :, 2]) 
    img_LMS[...,2] = np.log10(0.0241*img_RGB[:, :, 0] + 0.1288*img_RGB[:, :, 1] + 0.8444*img_RGB[:, :, 2])

    CIECAM97s_LMS=np.array([[2.00,1.00,0.05],[1.00,-1.09,0.09],[0.11,0.11,-0.22]])
    img_CIECAM97s=helper3by3(CIECAM97s_LMS,img_LMS)

    return img_CIECAM97s

In [ ]:

def convert_color_space_CIECAM97s_to_RGB(img_CIECAM97s):
    '''
    convert image color space CIECAM97s to RGB
    '''
    img_LMS = np.zeros_like(img_CIECAM97s,dtype=np.float32)
    CIECAM97s_LMS=np.array([[2.00,1.00,0.05],[1.00,-1.09,0.09],[0.11,0.11,-0.22]])
    LMS_CIECIA97s=np.linalg.inv(CIECAM97s_LMS)
    img_LMS=helper3by3(LMS_CIECIA97s,img_CIECAM97s)

    img_RGB = np.zeros_like(img_CIECAM97s,dtype=np.float32)
    img_LMS = np.power(10, img_LMS)
    RGB_LMS=np.array([[4.4679,-3.5873,0.1193],[-1.2186,2.3809,-0.1624],[0.0497,-0.2439,1.2045]])
    img_RGB=helper3by3(RGB_LMS,img_LMS)
    return img_RGB

In [ ]:
def transfer(img_source,img_target):
  # Split the images into their color channels
  s_channel0 = img_source[:, :, 0]
  s_channel1 = img_source[:, :, 1]
  s_channel2 = img_source[:, :, 2]
  t_channel0 = img_target[:, :, 0]
  t_channel1 = img_target[:, :, 1]
  t_channel2 = img_target[:, :, 2]  
  result=np.zeros_like(img_source,dtype=np.float32)
  # Calculate the mean and standard deviation of each channel
  t0mean=np.mean(t_channel0)
  t1mean=np.mean(t_channel1)
  t2mean=np.mean(t_channel2)
  s0mean=np.mean(s_channel0)
  s1mean=np.mean(s_channel1)
  s2mean=np.mean(s_channel2)
  t0std=np.std(t_channel0-t0mean)
  t1std=np.std(t_channel1-t1mean)
  t2std=np.std(t_channel2-t2mean)
  s0std=np.std(s_channel0-s0mean)
  s1std=np.std(s_channel1-s1mean)
  s2std=np.std(s_channel2-s2mean)
  # Transfer the color distribution of the source image to the target image
  for row in range(0,img_source.shape[0]):
            for col in range(0,img_source.shape[1]):
              result[row,col][0] = (t0std/s0std)*(img_source[row,col][0] - s0mean) + t0mean
              result[row,col][1] = (t1std/s1std)*(img_source[row,col][1] - s1mean) + t1mean
              result[row,col][2] = (t2std/s2std)*(img_source[row,col][2] - s2mean) + t2mean
  #print(result)
  return result
  

In [ ]:
def color_transfer_in_Lab(img_RGB_source, img_RGB_target):
    print('===== color_transfer_in_Lab =====')
    img_Lab_source=convert_color_space_RGB_to_Lab(img_RGB_source)
    img_Lab_target=convert_color_space_RGB_to_Lab(img_RGB_target)
    img_Lab=transfer(img_Lab_source,img_Lab_target)
    #print(img_Lab)
    img_RGB = convert_color_space_Lab_to_RGB(img_Lab)
    #img_RGB = np.clip(img_RGB, 0, 1)
    #print(img_RGB)
    return img_RGB

In [ ]:
def color_transfer_in_RGB(img_RGB_source, img_RGB_target):
    print('===== color_transfer_in_RGB =====')  
    img_RGB=transfer(img_RGB_source,img_RGB_target)
    #print(img_RGB)
    return img_RGB

In [ ]:
def color_transfer_in_CIECAM97s(img_RGB_source, img_RGB_target):
    print('===== color_transfer_in_CIECAM97s =====')
    img_CIECAM97s_source=convert_color_space_RGB_to_CIECAM97s(img_RGB_source)
    img_CIECAM97s_target=convert_color_space_RGB_to_CIECAM97s(img_RGB_target)
    img_CIECAM97s=transfer(img_CIECAM97s_source,img_CIECAM97s_target)
    #print(img_Lab)
    img_RGB = convert_color_space_Lab_to_RGB(img_CIECAM97s)
    #print(img_RGB)
    return img_RGB


In [ ]:
def color_transfer(img_RGB_source, img_RGB_target, option):
    if option == 'in_RGB':
        img_RGB_new = color_transfer_in_RGB(img_RGB_source, img_RGB_target)
    elif option == 'in_Lab':
        img_RGB_new = color_transfer_in_Lab(img_RGB_source, img_RGB_target)
    elif option == 'in_CIECAM97s':
        img_RGB_new = color_transfer_in_CIECAM97s(img_RGB_source, img_RGB_target)
    return img_RGB_new

In [ ]:

def rmse(apath,bpath):
    """
    This is the help function to get RMSE score.
    apath: path to your result
    bpath: path to our reference image
    when saving your result to disk, please clip it to 0,255:
    .clip(0.0, 255.0).astype(np.uint8))
    """
    a = cv2.imread(apath).astype(np.float32)
    b = cv2.imread(bpath).astype(np.float32)
    print(np.sqrt(np.mean((a-b)**2)))

In [ ]:
rmse('resi')

In [ ]:
if __name__ == "__main__":
    print('==================================================')
    print('PSU CS 410/510, Winter 2022, HW1: color transfer')
    print('==================================================')

    path_file_image_source = sys.argv[1]
    path_file_image_target = sys.argv[2]
    path_file_image_result_in_Lab = sys.argv[3]
    path_file_image_result_in_RGB = sys.argv[4]
    path_file_image_result_in_CIECAM97s = sys.argv[5]

    # ===== read input images
    # img_RGB_source: is the image you want to change the its color
    # img_RGB_target: is the image containing the color distribution that you want to change the img_RGB_source to (transfer color of the img_RGB_target to the img_RGB_source)
    img_BGR_target = cv2.imread(filename= path_file_image_target).astype(np.float32)/255.0
    img_BGR_source = cv2.imread(filename=path_file_image_source).astype(np.float32)/255.0
    img_RGB_source=convert_color_space_BGR_to_RGB(img_BGR_source)
    img_RGB_target=convert_color_space_BGR_to_RGB(img_BGR_target)
    img_RGB_new_Lab       = color_transfer(img_RGB_source, img_RGB_target, option='in_Lab')
    img_BGR_Lab=convert_color_space_RGB_to_BGR(img_RGB_new_Lab)
    cv2.imwrite(filename=path_file_image_result_in_Lab, img=(img_BGR_Lab * 255.0).clip(0.0, 255.0).astype(np.uint8))
    img_RGB_new_RGB       = color_transfer(img_RGB_source, img_RGB_target, option='in_RGB')
    # todo: save image to path_file_image_result_in_RGB
    img_BGR_RGB=convert_color_space_RGB_to_BGR(img_RGB_new_RGB)
    cv2.imwrite(filename=path_file_image_result_in_RGB, img=(img_BGR_RGB * 255.0).clip(0.0, 255.0).astype(np.uint8))
    img_RGB_new_CIECAM97s = color_transfer(img_RGB_source, img_RGB_target, option='in_CIECAM97s')
    # todo: save image to path_file_image_result_in_CIECAM97s
    img_BGR_CIECAM97s=convert_color_space_RGB_to_BGR(img_RGB_new_CIECAM97s)
    cv2.imwrite(filename=path_file_image_result_in_CIECAM97s, img=(img_BGR_CIECAM97s * 255.0).clip(0.0, 255.0).astype(np.uint8))

PSU CS 410/510, Winter 2022, HW1: color transfer


IndexError: ignored